In [1]:
import numpy as np
import pandas as pd
from scipy.constants import R
from scipy.optimize import root_scalar
import matplotlib.pyplot as plt
from typing import NamedTuple
from functools import partial

In [2]:
db = pd.read_csv('db.csv')

In [3]:
db.name.iloc[:10]

0      Methane
1       Ethane
2      Propane
3     i-Butane
4     n-Butane
5    i-Pentane
6    n-Pentane
7     n-Hexane
8    n-Heptane
9     n-Octane
Name: name, dtype: object

In [4]:
names = ['Methane', 'Ethane', 'Propane', 'i-Butane', 'n-Butane',
         'i-Pentane', 'n-Pentane', 'n-Hexane', 'n-Heptane', 'n-Octane']

In [5]:
components = db[db.name.isin(names)]
components = components.dropna(axis='columns')

In [6]:
def calculate_tsat(
        tc: np.ndarray,
        pc: np.ndarray,
        afactor: np.ndarray,
        p: float
) -> np.ndarray:
    tsat = tc / (1 - 3 * np.log(p/pc) / (np.log(10) * (7 + 7 * afactor)))
    return tsat

In [7]:
def xi_from_yi(
        yi: np.ndarray,
        ki: np.ndarray
) -> np.ndarray:
    return yi / ki

In [8]:
def calculate_ki(
        tc: np.ndarray,
        pc: np.ndarray,
        afactor: np.ndarray,
        t: float,
        p: float
) -> np.ndarray:
    ki = np.exp(
        np.log(pc / p) + np.log(10) 
        * (7/3) * (1 + afactor) * (1 - tc/t)
    )
    return ki

## PR params calculation

In [9]:
class PRComponentParams(NamedTuple):
    ki: np.ndarray
    alphai: np.ndarray
    ai: np.ndarray
    bi: np.ndarray

In [10]:
def calculate_pr_params_for_components(
        t: float,
        tc: np.ndarray,
        pc: np.ndarray,
        afactor: np.ndarray,
        r: float
) -> PRComponentParams:
    ki = .37464 + 1.54226 * afactor - .2699 * afactor ** 2
    alphai = (1 + ki * (1 - (t / tc) ** .5)) ** 2
    ai = .45724 * (r * tc) ** 2 * alphai / pc
    bi = .07780 * r * tc / pc
    params = PRComponentParams(
        ki, alphai, ai, bi
    )
    return params

In [11]:
class PRMixtureParams(NamedTuple):
    aij: np.ndarray
    a: float
    b: float
    a_: float
    b_: float

In [12]:
def calculate_pr_params_for_mixture(
        t: float,
        p: float,
        kij: np.ndarray,
        mf: np.ndarray,
        ai: np.ndarray,
        bi: np.ndarray,
        r: float,
) -> PRMixtureParams:
    aij = np.zeros((ai.shape[0], ai.shape[0]), dtype=float)
    a = 0
    for i, _ in enumerate(ai):
        for j, _ in enumerate(ai):
            aij[i, j] = (ai[i] * ai[j]) ** .5 * (1 - kij[i, j])
            a += aij[i, j] * mf[i] * mf[j]
    b = (bi * mf).sum()
    a_ = a * p / (r * t) ** 2
    b_ = b * p / (r * t)
    params = PRMixtureParams(
        aij, a, b, a_, b_
    )
    return params

In [13]:
def calculate_kij(vc: np.ndarray, n: int = 1) -> np.ndarray:
    """Chueh-Prausnitz correlation:
    https://wiki.whitson.com/eos/bips/

    Args:
        vc (np.ndarray): _description_
        n (int, optional): _description_. Defaults to 1.

    Returns:
        np.ndarray: _description_
    """
    vc_r3 = vc ** (1 / 3)
    numerator = np.multiply.outer(vc_r3, vc_r3) ** .5
    denominator = np.add.outer(vc_r3, vc_r3) / 2
    kij = 1 - (numerator / denominator) ** n
    return kij

#### Solving Cubic Equation

In [51]:
def solve_cubic_equation(
        a: float, b: float, kind: str = 'max'
) -> float | None:
    if kind not in ('max', 'min'):
        print(f'Parameter "kind" must be "min" or "max" not {kind}')
        return
    
    c0 = b ** 3 + b ** 2 - a * b
    c1 = a - 3 * b ** 2 - 2 * b
    c2 = b - 1

    q1 = c2 * c1 / 6 - c0 / 2 - c2 ** 3 / 27
    p1 = c2 ** 2 / 9 - c1 / 3
    d = q1 ** 2 - p1 ** 3

    if d >= 0:
        print(d, q1)
        return (
            (q1 + d ** .5) ** (1 / 3) 
            + (np.abs(q1 - d ** .5)) ** (1 / 3) - c2 / 3
        )

    
    t1 = q1 * 2 / p1 ** 3
    t2 = (1 - t1) ** .5 / t1 ** .5 * q1 / np.abs(q1)
    theta = np.arctan(t2)

    z0 = 2 * p1 ** .5 * np.cos(theta / 3) - c2 / 3
    z1 = 2 * p1 ** .5 * np.cos((theta + 2 * np.pi) / 3) - c2 / 3
    z2 = 2 * p1 ** .5 * np.cos((theta * 4 * np.pi) / 3) - c2 / 3
    roots = z0, z1, z2

    return max(roots) if kind == 'max' else min(roots)

In [52]:
def calculate_fugasity(
        z: float,
        bi: np.ndarray,
        b: float,
        b_: float,
        a_: float,
        mf: np.ndarray,
        aij: np.ndarray,
) -> np.ndarray:
    log_phi = np.zeros_like(bi)
    for i, _ in enumerate(bi):
        s = 0
        for j, _ in enumerate(bi):
            s += mf[j] * aij[i, j]
        log_phi[i] = (
            bi[i] / b * (z - 1) - np.log(z - b_)
            - a_ / (2 * 2 ** .5 * b_) * (2 * s / a_ - bi[i] / b)
            * np.log((z + (1 + 2 * .5) * b_) / (z + (1 - 2 ** .5) * b_))
        )
    return np.exp(log_phi)

In [53]:
class DewPointResults(NamedTuple):
    t_dew_point: float
    xi: np.ndarray

In [54]:
def Rashford_Rice(
        zi: np.ndarray, ki: np.ndarray,
        phase_state: int | None = None,
        bracket: tuple[float, float] | None = None,
        **root_scalar_kwargs
) -> float:

    def obj_fun(e: float, zi: np.ndarray, ki: np.ndarray) -> float:
        s = (zi * (ki - 1) / (1 + e * (ki - 1))).sum()
        return s

    f = partial(obj_fun, zi=zi, ki=ki)
    if not bracket:
        a, b = 0, 1
        bracket = a, b
    else:
        a, b = bracket

    if phase_state:
        if phase_state == 1:
            return 0

        if phase_state == 2:
            return 1

        if phase_state == 3:
            x0 = a

        elif phase_state == 4:
            x0 = .99999

        else:
            print('Phase state error')
            return np.nan
    else:
        x0 = (a + b) / 2

    if np.sign(f(a) * f(b)) < 0:
        sol = root_scalar(
            f, bracket=bracket, x0=x0, x1=b,
            method='bisect', **root_scalar_kwargs
        )
        return sol.root

    print('Rashford-Race: No Solutions')
    return np.nan

In [55]:
def calculate_x_from_z(
        zi: np.ndarray,
        e: float,
        ki: np.ndarray
) -> np.ndarray:
    if np.isclose(e, 1.):
        return np.zeros_like(zi)

    if np.isclose(e, .0):
        return zi

    x = zi / (1 + e * (ki - 1))
    return x

In [56]:
def calculate_y_from_z(
        zi: np.ndarray,
        xi: np.ndarray,
        e: float,
        ki: np.ndarray
) -> np.ndarray:
    if np.isclose(e, 1.):
        return zi

    if np.isclose(e, .0):
        return np.zeros_like(zi)

    y = xi * ki
    return y

In [57]:
def calculate_dew_point(
        z: np.ndarray,
        tc: np.ndarray,
        pc: np.ndarray,
        afactor: np.ndarray,
        vc: np.ndarray,
        t: float,
        p: float,
        xtol: float = 1e-5,
        nit: int = 25
) -> DewPointResults:
    ki = calculate_ki(
        tc=tc, pc=pc, afactor=afactor, t=t, p=p
    )
    e = Rashford_Rice(zi=z, ki=ki)
    x = calculate_x_from_z(zi=z, e=e, ki=ki)
    y = calculate_y_from_z(zi=z, xi=x, e=e, ki=ki)
    components_params = calculate_pr_params_for_components(
        t=t, tc=tc, pc=pc, afactor=afactor, r=R
    )
    kij = calculate_kij(vc)
    vparams = calculate_pr_params_for_mixture(
        t=t, p=p, kij=kij, mf=y, ai=components_params.ai,
        bi=components_params.bi, r=R
    )
    zv = solve_cubic_equation(
        a=vparams.a_, b=vparams.b_ 
    )
    phi_v = calculate_fugasity(
        z=zv, bi=components_params.bi,
        b=vparams.b, b_=vparams.b_, a_=vparams.a_,
        mf=y, aij=vparams.aij
    )

    x_prev = x
    for _ in range(nit):
        lparams = calculate_pr_params_for_mixture(
            t=t, p=p, kij=kij, mf=x, ai=components_params.ai,
            bi=components_params.bi, r=R
        )
        zl = solve_cubic_equation(
            a=lparams.a_, b=lparams.b_, kind='min'
        )
        phi_l = calculate_fugasity(
            z=zl, bi=components_params.bi,
            b=lparams.b, b_=lparams.b_, a_=lparams.a_,
            mf=y, aij=lparams.aij
        )
        x = y * phi_v / phi_l
        
        err = np.abs(x_prev - x)
        if err.sum() <= xtol:
            break
        x_prev = x

    return (ki * x).sum()

In [58]:
z = np.ones(len(names), dtype=float)
z /= z.sum()

In [59]:
tc = components['CriticalTemperatureValue'].to_numpy() + 273.15
pc = components['CriticalPressureValue'].to_numpy() / 100
afactor = components['AcentricityValue'].to_numpy()
vc = components['CriticalVolumeValue'].to_numpy()

In [60]:
p = 10.1325
t = 273.15

In [61]:
sol = calculate_dew_point(
    z=z, tc=tc, pc=pc, afactor=afactor,
    vc=vc, t=t, p=p
)

1.1383635260345747e-05 0.029253674796358986
0.0009163401117845059 -0.028656901350285438


C:\Users\Vyacheslav\AppData\Local\Temp\ipykernel_14000\3552197288.py:49: RuntimeWarning: invalid value encountered in divide
  x = y * phi_v / phi_l


In [63]:
sol

nan